In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import sys
sys.path.append('../')

from dataloader import get_model_values_df, sample_data
from utils import DATASET_TO_N_CLASSES 

run = 46
np.random.seed(run)
n_labeled = 20
n_unlabeled = 1000
dataset = 'civilcomments'
model_alg_combinations="distilbert_CORAL,distilbert_ERM,distilbert_IRM,distilbert_ERM_seed1,distilbert_ERM_seed2,distilbert_IRM_seed1,distilbert_IRM_seed2"
model_alg_list = model_alg_combinations.split(',')

group_definition = 'global'
num_groups = 1
n_classes = DATASET_TO_N_CLASSES[dataset]

split_name = 'labeled'
dataset_df = get_model_values_df(dataset, model_alg_list, split_name, value='prob_prediction')

train_dataset_df = dataset_df.sample(frac=.5, random_state=run)
test_dataset_df = dataset_df[~dataset_df.index.isin(train_dataset_df.index)]

# Sample data according to n_labeled and n_unlabeled
sampled_data, sampled_labels, sampled_true_labels, sampled_data_df = sample_data(train_dataset_df, n_labeled, n_unlabeled, model_alg_list, run, n_classes)
labeled_idxs = np.where(sampled_labels != -1)[0]
unlabeled_idxs = np.where(sampled_labels == -1)[0]

# Reorder sampled_data_df to match the order of estimation labeled and unlabeled data
sampled_data_df = pd.concat([sampled_data_df.iloc[labeled_idxs], sampled_data_df.iloc[unlabeled_idxs]], ignore_index=True)

heldout_data = test_dataset_df[model_alg_list].values
heldout_true_labels = test_dataset_df['label'].values

sampled_groups = [np.array(['global' for _ in range(n_labeled + n_unlabeled)])]
heldout_groups = [np.array(['global' for _ in range(len(test_dataset_df))])]

# Only useful when looking at a specific demographic, drawn from the metadata columns
subgroup = None
if group_definition == 'subgroups':
    sampled_groups.append(np.array(sampled_data_df[subgroup].values))
    heldout_groups.append(np.array(test_dataset_df[subgroup].values))

assert len(labeled_idxs) == n_labeled
assert len(unlabeled_idxs) == n_unlabeled


ModuleNotFoundError: No module named 'pandas'

In [ ]:
train_labeled_data = (sampled_data[labeled_idxs],
                            [d[labeled_idxs] for d in sampled_groups],
                            sampled_labels[labeled_idxs])

train_unlabeled_data = (sampled_data[unlabeled_idxs],
                                [d[unlabeled_idxs] for d in sampled_groups],
                                sampled_labels[unlabeled_idxs])
    
train_labeled_data_labels = sampled_true_labels[labeled_idxs]
train_unlabeled_data_labels = sampled_true_labels[unlabeled_idxs]

y_true = np.concatenate([train_labeled_data_labels, train_unlabeled_data_labels])

test_data = (heldout_data,
                heldout_groups,
                heldout_true_labels)

print("Prior: ", np.mean(train_unlabeled_data_labels), np.mean(sampled_true_labels))

In [ ]:
from model import SSME_KDE_binary

method_config = {'dataset': dataset, 
                 'epochs': 20, 
                 'labeled_data_weight': 10,
                 'simulate_predictions': True, 
                 'prior_type': 'learned', 
                 'use_sample_weights': True, 
                 'init': 'draw'}


SSME_KDE_binary(train_labeled_data, train_unlabeled_data, method_config)

In [ ]:
method_config = {'dataset': dataset, 
                'group_definition': group_definition,
                'model_alg_combinations': model_alg_combinations,}
gt_metrics_df = labeled_auc_auprc_ece(test_data, method_config)
gt_metrics_df